In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()


In [ ]:
import urllib.request
from azureml.core.model import Model

# Download model
from pathlib import Path

my_file = Path("./model.onnx")
if not my_file.exists():
    urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "./model.onnx")


In [ ]:

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment, CondaDependencies

env = Environment.from_pip_requirements(name="onnxruntime_env", file_path='./model_requirements.txt')
env.register(workspace=ws)


inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score_real.py')

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=1, auth_enabled=True
)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
service.delete()
